# import library for Deep Learning using Tensorflow (Training to be done on GPU)

In [9]:

import tensorflow as tf
import tflearn 


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


# Import library for preprocessing of data

In [1]:
import numpy as np
import pandas as pd
from PIL import ImageGrab
import cv2 as cv2
import keyboard
from tqdm import tqdm
import time
import os

# Testing the Screen grab and key grab here

In [15]:
while True:
    cv2.imshow('window',cv2.resize(cv2.cvtColor(np.array(ImageGrab.grab(bbox=(0,40,640,510))),cv2.COLOR_BGR2GRAY),(500,400)))
    if keyboard.is_pressed('W'):
        print("W pressed")
    if keyboard.is_pressed('S'):
        print("S pressed")    
    if cv2.waitKey(25) and 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

S pressed
S pressed
S pressed
W pressed
W pressed
W pressed
S pressed
S pressed
S pressed
W pressed
W pressed
S pressed
S pressed
S pressed
W pressed
W pressed
W pressed
W pressed
S pressed
S pressed
S pressed
S pressed
S pressed
S pressed
S pressed
S pressed
W pressed
W pressed
W pressed
S pressed
S pressed
W pressed
W pressed
W pressed
W pressed
W pressed
W pressed
W pressed
S pressed
S pressed
S pressed
S pressed
S pressed
W pressed
W pressed
W pressed
W pressed
S pressed
S pressed
S pressed
S pressed
S pressed
S pressed
S pressed
W pressed
W pressed
W pressed
W pressed
W pressed
W pressed
W pressed
W pressed
W pressed
W pressed
S pressed
S pressed
S pressed
S pressed
S pressed
S pressed
W pressed
W pressed
W pressed
S pressed
S pressed
W pressed
W pressed
W pressed
W pressed
S pressed
S pressed
S pressed
W pressed
W pressed
W pressed


KeyboardInterrupt: 

# preprocessing the data for training

In [ ]:
'''

The Preprocessing is done as follows : -

Grab the game screen with pillow library of box size = (0,40,640,510)
Use openCV to convert the image into Gray scale image.
Listen to the keyboard for input to the game (here W or S or nothing)
Save the key pressed in the form of 'one hot encoding' 
append the captured frame and key(one hot encoding) in a numpy array 
repeat n number of times

CAUTION !! careful what the n should be higher number might result in running out of memory.

reshape the data so as to have the frame and its respective one hot encoding in the same row for easy use later on.
Save the numpy array to disk so that you can use it later or have it as a dataset for the Retropong game.

'''

In [3]:
time.sleep(2)

array = []
key = []
frame = []
for i in tqdm(range(0,50000)):
    frame = np.array(cv2.resize(cv2.cvtColor( np.array(ImageGrab.grab(bbox=(0,40,640,510))),cv2.COLOR_BGR2GRAY),(100,100)))
    frame = frame.flatten()
    if keyboard.is_pressed('W'):
        key = np.array([1,0,0])
        array = np.append(array,(frame,key))
    elif keyboard.is_pressed('S'):
        key = np.array([0,1,0])
        array = np.append(array,(frame,key))  
    else:
        key = np.array([0,0,1])
        array = np.append(array,(frame,key))

array = np.array(array)
array = array.reshape(50000,2)
array.shape
np.save('data.npy',array)

100%|████████████████████████████████████████████████████████████████████████████| 50000/50000 [30:58<00:00, 26.91it/s]


# Start the Training on the Neural Network In Tensorflow

In [ ]:

n_classes = 3
batch_size = 16

x = tf.placeholder(tf.float32, [None, 200,200])
y = tf.placeholder(tf.float32, [None, 3])

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)


def cnn_and_dnn(x):
    with tf.device("/cpu:0"):
        weights = {'conv1_W':tf.Variable(tf.random_normal([5,5,1,32])),
                   'conv2_W':tf.Variable(tf.random_normal([5,5,32,64])),
                   'conv3_W':tf.Variable(tf.random_normal([5,5,64,128])),
                   'fc_1':tf.Variable(tf.random_normal([512,1024])),
                   'fc_2':tf.Variable(tf.random_normal([1024,512])),
                   'output':tf.Variable(tf.random_normal([512, n_classes]))}

        biases = {'conv1_B':tf.Variable(tf.random_normal([32])),
                   'conv2_B':tf.Variable(tf.random_normal([64])),
                   'conv3_B':tf.Variable(tf.random_normal([128])),
                   'fc_1':tf.Variable(tf.random_normal([1024])),
                   'fc_2':tf.Variable(tf.random_normal([512])),
                   'output':tf.Variable(tf.random_normal([n_classes]))}

        x = tf.reshape(x, shape=[-1, 100, 100, 1])
    
    with tf.device("/gpu:0"):
        conv1 = tf.nn.relu(tf.nn.conv2d(x, weights['conv1_W'], strides=[1,2,2,1], padding='SAME') + biases['conv1_B'])
        conv1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

        conv1 = tf.nn.relu(tf.nn.conv2d(conv1, weights['conv2_W'], strides=[1,2,2,1], padding='SAME') + biases['conv2_B'])
        conv1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

        conv1 = tf.nn.relu(tf.nn.conv2d(conv1, weights['conv3_W'], strides=[1,2,2,1], padding='SAME') + biases['conv3_B'])
        conv1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

        #fc = tf.reshape(conv1,[-1, 8192])
        poolShape = conv1.get_shape().as_list()

        fc = tf.reshape(conv1, [-1, poolShape[1] * poolShape[2] * poolShape[3]])

        fc = tf.nn.relu(tf.matmul(fc, weights['fc_1'])+biases['fc_1'])
        fc = tf.nn.dropout(fc, keep_rate)
        fc = tf.nn.relu(tf.matmul(fc, weights['fc_2'])+biases['fc_2'])
        fc = tf.nn.dropout(fc, keep_rate)

        output = tf.matmul(fc, weights['output'])+biases['output']

    return output

def train(x):
    prediction = cnn_and_dnn(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = prediction,labels = y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            array = np.load('data.npy')
            array = array.reshape(8000,2)
            num_mini_batches = int(array.size/batch_size)
            for _ in range(num_mini_batches):
                counter = 0
                if int(array.size) >= batch_size:
                    counter = batch_size
                else:
                    counter = int(array.size)
                
                epoch_x = np.empty([counter,100,100])
                epoch_y = np.empty([counter,3])
                
                for i in range(0,counter):
                    epoch_x[i] = array[i][0]
                    epoch_y[i] = array[i][1]
                
                
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
                
                array = array[counter:]

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        #correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        print("Training Done !!!")
        #accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        #print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

train(x)

# Load the necessary TFLearn Libraries.

In [10]:
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

# Neural Network training using TFlearn

In [10]:
#tf.global_variables_initializer()
tf.reset_default_graph()

IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'retropong-like-resnet-50000'

array = np.load('data.npy')
array = array.reshape(50000,2)

test = np.load('test.npy')
test = test.reshape(10000,2)

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')


# VGG 19 start here
'''
convnet = conv_2d(convnet, 16, 3, activation='relu')

convnet = conv_2d(convnet, 16, 3, activation='relu')

convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 32, 3, activation='relu')

convnet = conv_2d(convnet, 32, 3, activation='relu')

convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = conv_2d(convnet, 64, 3, activation='relu')

convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = conv_2d(convnet, 128, 3, activation='relu')

convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = conv_2d(convnet, 128, 3, activation='relu')

convnet = max_pool_2d(convnet, 3)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = fully_connected(convnet, 1024, activation='relu')
convnet = fully_connected(convnet, 256, activation='relu')

'''

#CNN and LSTM combined

'''
convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 128, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)
'''
'''
convnet = tf.reshape(convnet, shape=[-1, 4, 8])

convnet =  tflearn.lstm(convnet, 128, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 128, activation='relu')

'''
'''
convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.8)
'''

#MUlti CNN

'''
convnet = conv_2d(convnet, 16, 5, activation='relu')
convnet = conv_2d(convnet, 16, 5, activation='relu')
convnet = conv_2d(convnet, 16, 5, activation='relu')
convnet = conv_2d(convnet, 16, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
#convnet = max_pool_2d(convnet, 5)

convnet = tf.reshape(convnet, shape=[-1, 4, 8])

convnet =  tflearn.lstm(convnet, 256, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 256, activation='relu')
'''
'''
convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.9)

convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.9)

convnet = fully_connected(convnet, 256, activation='relu')
convnet = dropout(convnet, 0.9)

convnet = fully_connected(convnet, 128, activation='relu')
#convnet = dropout(convnet, 0.8)
'''

#Light-NET
'''


# Conv layers ------------------------------------
convnet = conv_2d(convnet, 32, 5, strides=2, activation='relu')
convnet = conv_2d(convnet, 32, 3, strides=1, activation='relu')
convnet = max_pool_2d(convnet, 3, strides=2)
convnet = conv_2d(convnet, 128, 3, strides=1, activation='relu')
convnet = max_pool_2d(convnet, 3, strides=2)


# Fully Connected Layers -------------------------
convnet1 = fully_connected(convnet, 128, activation='tanh')
convnet1 = dropout(convnet1, 0.5)
convnet2 = fully_connected(convnet, 128, activation='tanh')
convnet2 = dropout(convnet2, 0.5)
convnet = fully_connected(convnet1, 128, activation='tanh')
convnet = dropout(convnet, 0.5)
convnet = fully_connected(convnet2, 128, activation='tanh')
convnet = dropout(convnet, 0.5)
'''

#Resnet like structure.

convnet = conv_2d(convnet, 32, 2, activation='relu')
one = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
two=convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 128, 2, activation='relu')
three = conv_2d(convnet, 128, 2, activation='relu')
convnet = conv_2d(one, 128, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = conv_2d(two, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = conv_2d(three, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 256, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 128, activation='relu')

convnet = fully_connected(convnet, 3, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')




X = np.array([i[0] for i in array]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in array]


test_X = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_Y = [i[1] for i in test]


model.fit({'input': X}, {'targets': Y}, n_epoch=10, snapshot_step=500, show_metric=True, run_id=MODEL_NAME, validation_set=({'input': test_X}, {'targets': test_Y}))

model.save(MODEL_NAME)



Training Step: 23459  | total loss: 0.18613 | time: 201.855s
| Adam | epoch: 010 | loss: 0.18613 - acc: 0.9475 -- iter: 49984/50000
Training Step: 23460  | total loss: 0.17999 | time: 214.667s
| Adam | epoch: 010 | loss: 0.17999 - acc: 0.9512 | val_loss: 2.11440 - val_acc: 0.5076 -- iter: 50000/50000
--
INFO:tensorflow:C:\Users\Shekhar Shiroor\retropong-like-resnet-50000 is not in all_model_checkpoint_paths. Manually adding it.


# Load Trained Model and start playing the game.

In [16]:
tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'retropong-like-resnet-50000'

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
one = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
two=convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 128, 2, activation='relu')
three = conv_2d(convnet, 128, 2, activation='relu')
convnet = conv_2d(one, 128, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = conv_2d(two, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = conv_2d(three, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 512, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 256, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 128, activation='relu')

convnet = fully_connected(convnet, 3, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


while True:
    frame = np.array(cv2.resize(cv2.cvtColor( np.array(ImageGrab.grab(bbox=(0,40,640,510))),cv2.COLOR_BGR2GRAY),(100,100)))
    frame = frame.flatten()
    frame = frame.reshape(IMG_SIZE,IMG_SIZE,1)
    model_out = model.predict([frame])
    
    if np.argmax(model_out) == 0:
        keyboard.press('W')
        time.sleep(0.04)
        keyboard.release('W')
        #print(np.argmax(model_out))
        
    elif np.argmax(model_out) == 1:
        keyboard.press('S')
        time.sleep(0.04)
        keyboard.release('S')
        #print(np.argmax(model_out))
        
    else:
        #print(np.argmax(model_out))
        continue
        

model loaded!


KeyboardInterrupt: 

In [7]:
model = []

In [12]:
# another Variant 

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'retropong-cnn_lstm-train-50000'

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = tf.reshape(convnet, shape=[-1, 4, 8])

convnet =  tflearn.lstm(convnet, 128, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 128, activation='relu')


convnet = fully_connected(convnet, 3, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


while True:
    frame = np.array(cv2.resize(cv2.cvtColor( np.array(ImageGrab.grab(bbox=(0,40,640,510))),cv2.COLOR_BGR2GRAY),(100,100)))
    frame = frame.flatten()
    frame = frame.reshape(IMG_SIZE,IMG_SIZE,1)
    model_out = model.predict([frame])
    
    if np.argmax(model_out) == 0:
        keyboard.press('W')
        time.sleep(0.04)
        keyboard.release('W')
        #print(np.argmax(model_out))
        
    elif np.argmax(model_out) == 1:
        keyboard.press('S')
        time.sleep(0.04)
        keyboard.release('S')
        #print(np.argmax(model_out))
        
    else:
        #print(np.argmax(model_out))
        continue

model loaded!


KeyboardInterrupt: 

# Tensorboard embedding visualizer code

In [4]:
array = np.load('data.npy')
array = array.reshape(50000,2)



file = open( 'metadata.tsv', 'w')
file.write('Name\tClass\n')

for i in tqdm(range(0,2000)):
    temp = array[i][1]
    if(temp[0]==1):
        file.write('%s\n' % ('W'))
    elif(temp[1] == 1):
        file.write('%s\n' % ('S'))
    else:
        file.write('%s\n' % ('NA'))
        
file.close()        

#file.write('%06d\t%s\n' % (i,'NA'))

100%|██████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 306892.81it/s]


In [5]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

array = np.load('data.npy')
array = array.reshape(50000,2)

temp = []

for i in tqdm(range(0,2000)):
    temp = np.append(temp,array[i][0])

temp = temp.reshape(2000,10000)    

embedding_variable = tf.Variable(temp, name='embedding')

summary_writer = tf.summary.FileWriter('./log')
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_variable.name
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(summary_writer, config)


session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

saver = tf.train.Saver()
saver.save(session, './log/model.ckpt', 0)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:40<00:00, 19.94it/s]


'./log/model.ckpt-0'